In [21]:
def get_current_dir():
    import os
    from pathlib import Path
    try:
        return Path(__file__).parent.absolute()
    except NameError:
        return Path(os.getcwd())

path = get_current_dir().parent / "data" / "results.csv"

import pandas as pd
df = pd.read_csv(path)

# experiments from paper:
# - alpha_ent: 0.30 0.40
# - k: 20 50 100 200 (default=500 is not used in paper)
# - alpha_word: 0.2 0.3 0.4 0.5

# common values for undefined parameters:
# - n_topics: 10 20 50 100
# - n_neighbours: 5 10 20 500

missing = 0
for n_topics in [10, 20, 50, 100]:
    for n_neighbours in [5, 10, 20, 500]:
        for alpha_ent in [0.30, 0.40]:
            for k in [20, 50, 100, 200]:
                for alpha_word in [0.2, 0.3, 0.4, 0.5]:
                    mask = (df.n_topics == n_topics) & (df.n_neighbours == n_neighbours) & (df.alpha_ent == alpha_ent) & (df.k == k) & (df.alpha_word == alpha_word)
                    if not df[mask].empty:
                        continue
                    missing += 1
cycle_time_s = 600
print(f"missing: {missing}")
print(f"estimated time left: {missing * cycle_time_s // 3600}h {missing * cycle_time_s % 3600 // 60}m {missing * cycle_time_s % 60}s")

# print dataframe as latex table
print(df.to_latex(index=False))

missing: 0
estimated time left: 0h 0m 0s
\begin{tabular}{rrrrrrr}
\toprule
alpha_ent & k & n_topics & n_neighbours & alpha_word & score_deezer & score_itunes \\
\midrule
0.300000 & 20 & 10 & 5 & 0.200000 & 48.678100 & 51.668200 \\
0.300000 & 20 & 10 & 5 & 0.300000 & 48.678100 & 51.668200 \\
0.300000 & 20 & 10 & 5 & 0.400000 & 49.602300 & 50.995100 \\
0.300000 & 20 & 10 & 5 & 0.500000 & 49.856800 & 48.942800 \\
0.300000 & 20 & 10 & 10 & 0.200000 & 52.903800 & 51.369600 \\
0.300000 & 20 & 10 & 10 & 0.300000 & 52.903800 & 51.369600 \\
0.300000 & 20 & 10 & 10 & 0.400000 & 54.578000 & 52.581900 \\
0.300000 & 20 & 10 & 10 & 0.500000 & 52.162200 & 51.647200 \\
0.300000 & 20 & 10 & 20 & 0.200000 & 51.747600 & 51.406200 \\
0.300000 & 20 & 10 & 20 & 0.300000 & 51.747600 & 51.406200 \\
0.300000 & 20 & 10 & 20 & 0.400000 & 53.479300 & 50.824000 \\
0.300000 & 20 & 10 & 20 & 0.500000 & 54.610600 & 50.142200 \\
0.300000 & 20 & 10 & 500 & 0.200000 & 47.911800 & 51.432600 \\
0.300000 & 20 & 10 & 500 & 